### Proyecto 6

Regresión logística y regresión lineal

Como siempre comenzamos preparando los datos. Primer instalamos la librería que nos permitirá su descarga. 


In [ ]:
modules = ["ucimlrepo"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules) 

En este caso son  datos de crecimiento de plantas de soja. Lo detalles de las columnas se pueden ver [aquí](https://archive.ics.uci.edu/dataset/913/forty+soybean+cultivars+from+subsequent+harvests) (ver tabla en la parte de abajo de la página). La carga es un poco particular porque usamos la API de la universidad de Irvine, pero resultado es un dataframe `df` como siempre:

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
forty_soybean_cultivars_from_subsequent_harvests = fetch_ucirepo(id=913) 
  
# data (as pandas dataframes) 
df = forty_soybean_cultivars_from_subsequent_harvests.data.features 


df = df.drop(columns=["Cultivar"])
df

In [ ]:
df.describe()

**Ejercicio 1** 

Queremos  obtener la altura de la planta (columna `PH`) a partir del resto. Podemos tomar como punto de partida el notebook [proyecto4_sol](https://github.com/RafaelCaballero/BME/blob/main/c%C3%B3digo/proyecto_4_sol.ipynb)

In [ ]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import math

#### Solución



¿Qué rango de error tendrán las predicciones que hagamos?

**Ejercicio 2** Nos damos cuenta de que realmente no necesitamos una regresión, porque solo queremos dstinguir entre dos tipos de altura: planteas altas (1) y plantas bajas, (0), tomando com punto de división la mediana. Podemos construir una columna `label` a partir de `PH`:

In [ ]:
import pandas as pd
forty_soybean_cultivars_from_subsequent_harvests = fetch_ucirepo(id=913) 
  
# data (as pandas dataframes) 
df = forty_soybean_cultivars_from_subsequent_harvests.data.features 

df['label'] = pd.qcut(df["PH"], q=2, labels=[ 0, 1]) # pd.qcut genera el valor 0 para la altura por debajo de la mediana y 1 por encima.
df = df.drop(columns=["Cultivar","PH"]) # PH se sustituye por label
df

Probar un clasificador basado en regresión logística para determinar `label` a partir del resto de columnas. Para ver los errores podemos utilizar la siguiente función

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, accuracy_score, recall_score,cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# para mostrar las funciones de error. Se le pasan os valores reales (los del test) y los predicho
def show_results(y_real,y_pred,y_pred_proba):
    print("\nKappa ", round(cohen_kappa_score(y_real,y_pred),4) )
    cm = confusion_matrix(y_real, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=modelo.classes_)
    disp.plot()
    
    plt.show()
    print('\nClassification Report: \n', classification_report(y_real, y_pred))
    return


El código para hacer la regresión se puede extraer de [15logistica](https://github.com/RafaelCaballero/BME/blob/main/c%C3%B3digo/15logistica.ipynb), en la casilla donde se hacen los 4 pasos, pero adaptándolos. En particular recordar que la y ahora es `label`, que el dataframe se llama `df`. Poner el número de iteraciones de la regresión logística (parámetro `max_iter`) a  10000


In [ ]:
from sklearn.linear_model import LogisticRegression

# solución; ver los pasos 1,2,3,4 del notebook 15logistica, en la celda en la que se aplica la regresión logística, 
# pero recordar que aquí no es df2, sino df, y que la columna a predecir es 'label' y no 'survived' 


Intentar interpretar, en la medida de lo posible los resultados ¿son buenos?